In [5]:
import os
import openai
from dotenv import load_dotenv
import pandas as pd
load_dotenv()
openai.api_key = os.environ.get("OPENAI_API_KEY")

In [2]:
def get_label(df):
    label = []
    broken = []
    for i in range(len(df)):
        paragraph = df[i]
        if(i == len(df) // 2):
            print(i)
        # time.sleep(2)
        try:
            completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "The assistant is a ESG-investing field advisor,choose at most top three related keys of the paragraph. Return the corresponding values each range from 0 to 26 using LDA_table dictionary below."\
                "LDA_table dictionary below constructed by {'key': value},return only the value without key."
                "Do not return the key of the LDA_table dictionary."
                "LDA_table dictionary:\
                {'Business ethics': 0, 'Data security': 1, 'Access_And_Affordability': 2, 'Business_Model_Resilience': 3, 'Competitive_Behavior': 4,\
                'Critical_Incident_Risk_Management': 5, 'Customer_Welfare': 6, 'Director_Removal': 7, 'Employee_Engagement_Inclusion_And_Diversity': 8,\
                'Employee_Health_And_Safety': 9, 'Human_Rights_And_Community_Relations': 10, 'Labor_Practices': 11, 'Management_Of_Legal_And_Regulatory_Framework': 12,\
                'Physical_Impacts_Of_Climate_Change': 13, 'Product_Quality_And_Safety': 14, 'Product_Design_And_Lifecycle_Management': 15,\
                'Selling_Practices_And_Product_Labeling': 16, 'Supply_Chain_Management': 17, 'Systemic_Risk_Management': 18,\
                'Waste_And_Hazardous_Materials_Management': 19, 'Water_And_Wastewater_Management': 20, 'Air_Quality': 21,\
                'Customer_Privacy': 22, 'Ecological_Impacts': 23, 'Energy_Management': 24, 'GHG_Emissions': 25, 'No_sufficient_information': 26}"},
                {"role": "user", "content": f"paragraph: {paragraph}"},
                {"role": "assistant", "content": "[value]"},
            ],
            max_tokens = 10
            )
        except Exception as e:
            broken.append(i)
            print(i, e)
        else:
            label.append(completion.choices[0].message.content)
            # print(df[i], completion.choices[0].message.content)
    return label, broken

In [3]:
def fill_fail_sentence(label, broken, df):
    while(len(broken)):
        broke_idx = broken[0]
        paragraph = df[broke_idx]
        del broken[0]
        try:
            completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "The assistant is a ESG-investing field advisor,choose at most top three related keys of the paragraph. Return the corresponding values each range from 0 to 26 using LDA_table dictionary below."\
                "LDA_table dictionary below constructed by {'key': value},return only the value without key."
                "Do not return the key of the LDA_table dictionary."
                "LDA_table dictionary:\
                {'Business ethics': 0, 'Data security': 1, 'Access_And_Affordability': 2, 'Business_Model_Resilience': 3, 'Competitive_Behavior': 4,\
                'Critical_Incident_Risk_Management': 5, 'Customer_Welfare': 6, 'Director_Removal': 7, 'Employee_Engagement_Inclusion_And_Diversity': 8,\
                'Employee_Health_And_Safety': 9, 'Human_Rights_And_Community_Relations': 10, 'Labor_Practices': 11, 'Management_Of_Legal_And_Regulatory_Framework': 12,\
                'Physical_Impacts_Of_Climate_Change': 13, 'Product_Quality_And_Safety': 14, 'Product_Design_And_Lifecycle_Management': 15,\
                'Selling_Practices_And_Product_Labeling': 16, 'Supply_Chain_Management': 17, 'Systemic_Risk_Management': 18,\
                'Waste_And_Hazardous_Materials_Management': 19, 'Water_And_Wastewater_Management': 20, 'Air_Quality': 21,\
                'Customer_Privacy': 22, 'Ecological_Impacts': 23, 'Energy_Management': 24, 'GHG_Emissions': 25, 'No_sufficient_information': 26}"},
                {"role": "user", "content": f"paragraph: {paragraph}"},
                {"role": "assistant", "content": "[value]"},
            ],
            max_tokens = 10
            )
        except Exception as e:
            print(e)
            broken.append(broke_idx)
        else:
            label.insert(broke_idx, completion.choices[0].message.content)
            print(broke_idx, completion.choices[0].message.content)
    return label

In [ ]:
file_list = ['1_dow_1_85.pdf',
'2_agilent_1_106.pdf', '3_amazon_1_79.pdf', '4_apple_1_72.pdf',
'5_boeing_1_66.pdf', '6_bxp_1_65.pdf', '7_charles_1_50.pdf', '8_cisco_1_56.pdf', '9_citigroup_1_137.pdf', '10_cme_1_34.pdf', 
'11_colgate_1_84.pdf', '12_corning_1_71.pdf', '13_expeditor_1_37.pdf', '14_eei_1_80.pdf', '15_itt_1_44.pdf', 
'16_fedex_1_34.pdf', '17_firstscolar_1_57.pdf', '18_google_1_14.pdf', '19_intel_1_86.pdf', '20_jpmorgan_1_61.pdf', 
'21_microsoft_1_89.pdf', '22_rockwell_1_58.pdf', '23_ibm_1_49.pdf', '24_traveler_1_147.pdf', '25_visa_1_52.pdf']

In [6]:
file_list = ['1_umc_1_154.pdf',
 '2_tsmc_1_210.pdf', '3_macronix_1_114.pdf', '4_esun_1_154.pdf', '5_eme_1_46.pdf',
 '6_asus_1_108.pdf', '7_acer_1_127.pdf', '8_witron_1_93.pdf', '9_honhai_1_22.pdf', '10_compal_1_169.pdf',
  '11_quanta_1_123.pdf', '12_formosa_1_18.pdf', '14_qisda_1_134.pdf', '15_msi_1_97.pdf',
 '18_novatek_1_140.pdf', '19_mediatek_1_97.pdf', 
   '21_cathay_1_52.pdf', '23_fubon_1_89.pdf', '24_mega_1_68.pdf', '25_ctbc_1_117.pdf']

In [7]:
# for check model effect delete later
file_name = 'apple_3&gpt_label'
df = pd.read_csv(f'{file_name}.csv')
paragraph = df['paragraph']
label, broken = get_label(paragraph)
# print(label)
# label = fill_fail_sentence(label, broken, paragraph)
# new_label = pd.Series(label, name='GPTOutput_label', dtype='int64')
# ans = pd.concat([df, new_label], axis=1)
# ans.to_csv(f'apple_4label_keywords_expalin.csv', index=False)

across our environmental work we are also committed to being a force for equity were working with communities on the front lines of climate change and the next generation of diverse entrepreneurs to create shared opportunity and to build a more just world [10, 8, 23]
the environmental challenges we face today are significant and we are responding with urgency and dedication we approach our work by focusing on fundamental questions what matters most and where can we make the greatest impact these questions guide our work across our strategic focus areas of climate change resources and smarter chemistry 13, 14, 25


In [7]:
for file in file_list:
    print(file)
    file_name = file.strip('.pdf')
    df = pd.read_csv(f'./csv/xml_taiwan/{file_name}.csv')
    df = df['paragraph']
    label, broken = get_label(df)
    label = fill_fail_sentence(label, broken, df)
    ans = pd.DataFrame({'paragraph': df, 'label': label})
    ans.to_csv(f'./csv/predict_taiwan_3label/{file_name}.csv', index=False)

1_umc_1_154.pdf
101 Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600)
418 Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600)
833 Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600)
850
101 8, 10, 26
418 11
833 3, 8, 10
2_tsmc_1_210.pdf
578 Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600)
931
578 [0, 8, 23]
3_macronix_1_114.pdf
361
4_esun_1_154.pdf
105 Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600)
196 Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600)
427 Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600)
743
105 [2, 4, 17]
196 [17, 0, 26]
427 [3, 23]
5_eme_1_46.pdf
84
6_asus_1_108.pdf
21

# Appendix